# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234631692270                   -0.50    7.0
  2   -7.250220395215       -1.81       -1.40    1.0
  3   -7.251160877642       -3.03       -2.19    1.0
  4   -7.251310409150       -3.83       -2.26    2.0
  5   -7.251337059523       -4.57       -2.82    1.0
  6   -7.251337903340       -6.07       -3.09    2.0
  7   -7.251338702470       -6.10       -3.61    1.0
  8   -7.251338741469       -7.41       -3.77    2.0
  9   -7.251338784507       -7.37       -4.06    1.0
 10   -7.251338798013       -7.87       -4.65    1.0
 11   -7.251338798587       -9.24       -5.08    2.0
 12   -7.251338798692       -9.98       -5.53    2.0
 13   -7.251338798703      -10.95       -6.00    2.0
 14   -7.251338798704      -11.99       -6.36    1.0
 15   -7.251338798705      -12.57       -6.90    2.0
 16   -7.251338798705      -13.60       -7.17    3.0
 17   -7.251338798705      -14.27       -7.75 

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.034263735234470204
[ Info: Arnoldi iteration step 2: normres = 0.6049351305681692
[ Info: Arnoldi iteration step 3: normres = 0.9481044761800939
[ Info: Arnoldi iteration step 4: normres = 0.30452275088450936
[ Info: Arnoldi iteration step 5: normres = 0.41308930078652606
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.94e-02, 6.45e-02, 3.52e-01, 1.94e-01, 5.69e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3863120507700348
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.35e-02, 2.13e-01, 2.87e-01, 1.25e-01, 3.32e-02)
[ Info: Arnoldi iteration step 7: normres = 0.10953463585432946
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (7.67e-04, 2.94e-02, 1.72e-02, 6.50e-02, 7.29e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09742917551687891
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (3.22